In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [21]:
import logging

import duckdb
import holoviews as hv
import hvplot.polars
import neurokit2 as nk
import pandas as pd
import polars as pl
from icecream import ic
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.eda import nk_process_eda
from src.features.scaling import scale_min_max
from src.features.transformations import map_trials
from src.log_config import configure_logging
from src.visualization.utils import prepare_multiline_hvplot

configure_logging(
    stream_level=logging.DEBUG, ignore_libs=("Comm", "bokeh", "tornado", "matplotlib")
)
logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [22]:
db = DatabaseManager()

In [23]:
with db:
    df = db.read_table("Raw_eda")
df = df.unique("timestamp").sort("timestamp")  # sample_rate = 100 Hz

## Comparing neurokit / biopac / highpass filter (0.05 Hz) vs Median smoothing

From the neurokit2 documentation:
  * **High-pass filtering**: Method implemented in Biopac's Acqknowledge. The raw EDA signal
    is passed through a high pass filter with a cutoff frequency of 0.05 Hz
    (cutoff frequency can be adjusted by the ``cutoff`` argument).
  * **Median smoothing**: Method implemented in Biopac's Acqknowledge. The raw EDA signal is
    passed through a median value smoothing filter, which removes areas of rapid change. The
    phasic component is then calculated by subtracting the smoothed signal from the original.
    This method is computationally intensive and the processing time depends on the smoothing
    factor, which can be controlled by the as ``smoothing_factor`` argument, set by default to
    ``4`` seconds. Higher values will produce results more rapidly.

-> high pass filter is much faster than median smoothing

In [24]:
scale_min_max(
    nk_process_eda(
        df,
        method="neurokit",
    )
).plot(
    x="timestamp",
    y=["eda_raw", "eda_phasic", "eda_tonic"],
    groupby="trial_id",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'91f1e3ab-3b33-40a7-8ad1-12a078312c19': {'version…

In [6]:
# median smoothing is much slower, that why we only use it for one trial
trial_id = 98

scale_min_max(
    nk_process_eda(
        df.filter(col("trial_id") == trial_id),
        method="median",
    )
).plot(
    x="timestamp",
    y=["eda_raw", "eda_phasic", "eda_tonic"],
    groupby="trial_id",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'a7d2fd9d-11fc-4380-9a68-a5537825b3c6': {'version…

## Other available methods:
- convex
- sparse

In [7]:
# convex smoothing is even slower
trial_id = 98

scale_min_max(
    nk_process_eda(
        df.filter(col("trial_id") == trial_id),
        method="convex",
    )
).plot(
    x="timestamp",
    y=["eda_raw", "eda_phasic", "eda_tonic"],
    groupby="trial_id",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'cb15c415-3217-44ee-8999-f19d5ef6876a': {'version…

In [8]:
# sparse smoothing is even slower
trial_id = 98

scale_min_max(
    nk_process_eda(
        df.filter(col("trial_id") == trial_id),
        method="sparse",
    )
).plot(
    x="timestamp",
    y=["eda_raw", "eda_phasic", "eda_tonic"],
    groupby="trial_id",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'887a47fd-6051-49f0-a342-90eecfee38f0': {'version…

## Join with Stimulus data

In [26]:
query = """
SELECT * FROM Preprocess_EDA AS pe
ASOF JOIN Raw_Stimulus rs USING (trial_id, timestamp)
ORDER BY pe.trial_id, pe.timestamp
"""

with db:
    df = db.execute(query).pl()
df
ic(df.height, df.unique("timestamp").height)
# ensure that the timestamps are unique
ic(df.height - df.unique("timestamp").height)
df

ic| df.height: 5934569, df.unique("timestamp").height: 5934569
ic| df.height - df.unique("timestamp").height: 0


trial_id,trial_number,participant_id,rownumber,timestamp,samplenumber,eda_d_battery,eda_raw,eda_d_packetreceptionrate,temperature,rating
u16,u8,u8,u32,f64,i64,f64,f64,i64,f64,f64
1,1,1,0,294224.331,57895,3677.435897,0.753247,100,45.75,42.5
1,1,1,0,294242.275,57896,3687.630769,0.754135,100,45.75,42.5
1,1,1,0,294248.2588,57898,3692.0,0.754135,100,45.75,42.5
1,1,1,0,294276.1835,57899,3678.892308,0.754135,100,45.75,42.5
1,1,1,0,294277.1819,57900,3692.0,0.752359,100,45.75,42.5
1,1,1,0,294309.0952,57902,3705.107692,0.752359,100,45.75,42.5
…,…,…,…,…,…,…,…,…,…,…
332,12,28,21610,2.7771e6,467073,3603.158974,13.679468,99,45.582614,85.0
332,12,28,21610,2.7771e6,467074,3617.723077,13.679468,99,45.582614,85.0
